<a href="https://colab.research.google.com/github/Vedmani/wear-particle-classification/blob/main/EfficientNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install wandb -qU
!wandb login 617d80dc70f383d8ff9ef1c2b94a78ccc8d8db23

     |████████████████████████████████| 1.9 MB 35.1 MB/s 
     |████████████████████████████████| 182 kB 63.4 MB/s 
     |████████████████████████████████| 162 kB 65.3 MB/s 
     |████████████████████████████████| 63 kB 1.5 MB/s 
     |████████████████████████████████| 162 kB 65.8 MB/s 
     |████████████████████████████████| 158 kB 74.5 MB/s 
     |████████████████████████████████| 157 kB 72.8 MB/s 
     |████████████████████████████████| 157 kB 74.5 MB/s 
     |████████████████████████████████| 157 kB 62.5 MB/s 
     |████████████████████████████████| 157 kB 74.6 MB/s 
     |████████████████████████████████| 157 kB 76.6 MB/s 
     |████████████████████████████████| 157 kB 77.2 MB/s 
     |████████████████████████████████| 157 kB 76.9 MB/s 
     |████████████████████████████████| 156 kB 68.2 MB/s 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
import wandb
from wandb.keras import WandbCallback

In [3]:
tf.keras.utils.set_random_seed(123)
tf.config.experimental.enable_op_determinism()

In [4]:
PATH = '/content/drive/MyDrive/Research Internship/wear particles'
batch_size = 32
img_height = 224
img_width = 224

train_ds = tf.keras.utils.image_dataset_from_directory(
  PATH,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

val_ds = tf.keras.utils.image_dataset_from_directory(
  PATH,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

class_names = train_ds.class_names
print(class_names)
num_classes = len(class_names)

Found 740 files belonging to 5 classes.
Using 592 files for training.
Found 740 files belonging to 5 classes.
Using 148 files for validation.
['cutting wear', 'normal rubbing and cutting wear combined', 'normal rubbing wear', 'red oxide', 'severe sliding wear']


In [5]:
efficientnet = tf.keras.applications.EfficientNetB7(
    include_top=False,
    weights="imagenet",
    input_tensor=None,
    input_shape=None,
    pooling=None,
    classes=1000,
    classifier_activation="softmax",
)
data_augmentation = tf.keras.Sequential([layers.RandomFlip("horizontal_and_vertical"),layers.RandomRotation(0.2),])
model_efficientnet=Sequential([
  #data_augmentation,
  layers.Rescaling(1./255, input_shape=(img_height, img_width, 3)),
  efficientnet,
  layers.Flatten(),
  layers.Dense(32, activation='relu'),
  #layers.Dropout(0.5), 
  layers.Dense(num_classes, activation='softmax')
])
efficientnet.trainable=False
model_efficientnet.summary()

258076736/258076736 [==============================] - 1s 0us/step
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling_1 (Rescaling)     (None, 224, 224, 3)       0         
                                                                 
 efficientnetb7 (Functional)  (None, None, None, 2560)  64097687 
                                                                 
 flatten (Flatten)           (None, 125440)            0         
                                                                 
 dense (Dense)               (None, 32)                4014112   
                                                                 
 dense_1 (Dense)             (None, 5)                 165       
                                                                 
Total params: 68,111,964
Trainable params: 4,014,277
Non-trainable params: 64,097,687
_________________________________

In [6]:
model_efficientnet.compile(
              optimizer= tf.keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-07, amsgrad=False, name='Adam'),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
              metrics=['accuracy']
              )

In [7]:
wandb.init(
      # Set the project where this run will be logged
      project="Transfer_learning", 
      name = "efficientnetB7",
      notes = ""
)

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: vedmanivaidya. Use `wandb login --relogin` to force relogin


In [8]:
%%time
epochs=20
history = model_efficientnet.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs,
callbacks=[WandbCallback()]
)

wandb: WARNING The save_model argument by default saves the model in the HDF5 format that cannot save custom objects like subclassed models and custom layers. This behavior will be deprecated in a future release in favor of the SavedModel format. Meanwhile, the HDF5 model is saved as W&B files and the SavedModel as W&B Artifacts.


Epoch 1/20
19/19 [==============================] - ETA: 0s - loss: 13.6755 - accuracy: 0.2703

wandb: Adding directory to artifact (/content/wandb/run-20221020_054935-uvnh48hn/files/model-best)... Done. 1.9s


19/19 [==============================] - 374s 15s/step - loss: 13.6755 - accuracy: 0.2703 - val_loss: 1.6115 - val_accuracy: 0.1081
Epoch 2/20
19/19 [==============================] - ETA: 0s - loss: 1.6090 - accuracy: 0.1655

wandb: Adding directory to artifact (/content/wandb/run-20221020_054935-uvnh48hn/files/model-best)... Done. 2.0s


...

19/19 [==============================] - 190s 10s/step - loss: 1.4984 - accuracy: 0.4916 - val_loss: 1.5086 - val_accuracy: 0.4662
Epoch 19/20
19/19 [==============================] - ETA: 0s - loss: 1.4929 - accuracy: 0.4916

wandb: Adding directory to artifact (/content/wandb/run-20221020_054935-uvnh48hn/files/model-best)... Done. 1.9s


19/19 [==============================] - 194s 11s/step - loss: 1.4929 - accuracy: 0.4916 - val_loss: 1.5041 - val_accuracy: 0.4662
Epoch 20/20
19/19 [==============================] - ETA: 0s - loss: 1.4876 - accuracy: 0.4916

wandb: Adding directory to artifact (/content/wandb/run-20221020_054935-uvnh48hn/files/model-best)... Done. 2.0s


19/19 [==============================] - 187s 10s/step - loss: 1.4876 - accuracy: 0.4916 - val_loss: 1.4996 - val_accuracy: 0.4662
CPU times: user 1h 1min 26s, sys: 1min 56s, total: 1h 3min 22s
Wall time: 1h 9min 38s


In [9]:
wandb.finish()

accuracy,▃▁██████████████████
epoch,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇██
loss,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_accuracy,▁███████████████████
val_loss,██▇▇▆▆▆▅▅▄▄▄▃▃▃▂▂▂▁▁
accuracy,0.49155
best_epoch,19
best_val_loss,1.49955
epoch,19
loss,1.4876
val_accuracy,0.46622
